In [1]:
import numpy as np
from datascience import *
import pandas as pd

import os
import re
import json

d:\VNU\Lab\Tools\.venv\Lib\site-packages\datascience\maps.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Phân tích .json

In [ ]:
import json
import os
from urllib.parse import urlparse

# --- 1. Định nghĩa đường dẫn file ---
# Đảm bảo đường dẫn này khớp với vị trí của file sqli.json của bạn
zap_json_file_path = 'sqli.json' 
output_directory = 'founded_testcase'
output_file_name = 'cmdi_2_testcase.txt' # Đổi tên file output cho phù hợp với việc tìm kiếm nhiều loại

# Xây dựng đường dẫn đầy đủ cho file đầu ra
output_full_path = os.path.join(output_directory, output_file_name)

# --- 2. Đảm bảo thư mục đầu ra tồn tại ---
os.makedirs(output_directory, exist_ok=True)

# --- 3. Đọc dữ liệu từ file JSON của ZAP ---
zap_data = {}
try:
    with open(zap_json_file_path, 'r', encoding='utf-8') as f:
        zap_data = json.load(f)
    print(f"Đã đọc thành công file: {zap_json_file_path}")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{zap_json_file_path}'. Vui lòng kiểm tra lại đường dẫn.")
    exit()
except json.JSONDecodeError:
    print(f"Lỗi: Không thể giải mã JSON từ file '{zap_json_file_path}'. Đảm bảo đây là file JSON hợp lệ.")
    exit()

# --- 4. Xử lý dữ liệu và trích xuất tên test case ---
found_testcases = set() # Dùng set để tránh trùng lặp

# Định nghĩa danh sách các tên alert mục tiêu
target_alert_names = [
    "SQL Injection",
    "SQL Injection - Hypersonic SQL"
]

# Cờ để kiểm tra xem ít nhất một trong các alert mục tiêu có được tìm thấy không
any_target_alert_found_in_report = False 

# Kiểm tra và duyệt qua danh sách 'site'
if isinstance(zap_data, dict) and 'site' in zap_data and isinstance(zap_data['site'], list):
    for site_obj in zap_data['site']:
        if isinstance(site_obj, dict) and 'alerts' in site_obj:
            alerts_list = site_obj['alerts']
            
            if isinstance(alerts_list, list):
                for alert in alerts_list:
                    # Kiểm tra nếu tên alert hiện tại nằm trong danh sách các alert mục tiêu
                    if alert.get('alert') in target_alert_names:
                        any_target_alert_found_in_report = True
                        print(f"\nĐang xử lý alert: '{alert.get('alert')}' (pluginid: {alert.get('pluginid')})") # In ra alert đang xử lý

                        instances = alert.get('instances', [])
                        if isinstance(instances, list):
                            for instance in instances:
                                uri = instance.get('uri')
                                if uri:
                                    parsed_uri = urlparse(uri)
                                    path_segments = parsed_uri.path.split('/')
                                    
                                    testcase_name = None
                                    for segment in path_segments:
                                        if segment.startswith('BenchmarkTest'):
                                            testcase_name = segment.split('.')[0]
                                            break
                                    
                                    if testcase_name:
                                        found_testcases.add(testcase_name)
                            
                            # Cảnh báo nếu không có instance hoặc URI hợp lệ trong alert này
                            if not found_testcases and not alert.get('instances'):
                                print(f"Cảnh báo: Alert '{alert.get('alert')}' được tìm thấy nhưng không có 'instances' nào chứa URI hợp lệ.")
                        else:
                            print(f"Cảnh báo: Alert '{alert.get('alert')}' được tìm thấy nhưng phần 'instances' không phải là danh sách.")
else:
    print("Lỗi: Cấu trúc JSON không có key 'site' ở cấp cao nhất hoặc 'site' không phải là danh sách.")

# In thông báo tổng quan về alert đã tìm thấy (nếu có)
if any_target_alert_found_in_report:
    print(f"\nĐã xử lý các alert mục tiêu.")
else: # Chỉ in nếu cấu trúc 'site' tồn tại nhưng không tìm thấy alert nào trong danh sách mục tiêu
    print(f"Không tìm thấy bất kỳ alert nào trong danh sách {target_alert_names} trong báo cáo.")


# --- 5. Lưu các test case tìm được vào file ---
if found_testcases:
    with open(output_full_path, 'w', encoding='utf-8') as f:
        for tc in sorted(list(found_testcases)): # Sắp xếp để dễ đọc
            f.write(tc + '\n')
    print(f"\nĐã lưu {len(found_testcases)} tên test case vào file: {output_full_path}")
else:
    print(f"\nKhông tìm thấy test case nào cho các loại alert {target_alert_names} để lưu.")

Đã đọc thành công file: sqli.json

Đang xử lý alert: 'SQL Injection' (pluginid: 40018)

Đang xử lý alert: 'SQL Injection - Hypersonic SQL' (pluginid: 40018)

Đã xử lý các alert mục tiêu.

Đã lưu 170 tên test case vào file: found_testcase\cmdi_2_testcase.txt


## Đọc expected result 


In [3]:
# Đọc CSV và chỉ lấy 4 cột đầu
expected = Table.read_table("D:/VNU/Lab/Tools/expected_results.csv").select(0, 1, 2, 3)
# Đổi tên cột cho chuẩn: xóa khoảng trắng thừa
expected = (expected
            .relabeled("# test name", "test_name")
            .relabeled(" category", "category")
            .relabeled(" real vulnerability", "real_vulnerability")
            .relabeled(" cwe", "cwe")
           )

# Kiểm tra kết quả
expected.show(5)

test_name,category,real_vulnerability,cwe
BenchmarkTest00001,pathtraver,True,22
BenchmarkTest00002,pathtraver,True,22
BenchmarkTest00003,hash,True,328
BenchmarkTest00004,trustbound,True,501
BenchmarkTest00005,crypto,True,327


## Tách expected theo các category 

In [4]:
# Lấy tất cả category duy nhất
categories = np.unique(expected.column("category"))

# Tạo dictionary: key=category, value=bảng con
category = {cat: expected.where("category", cat) for cat in categories}

print("🔎 Danh sách các category đã tạo bảng con:")
for cat in category.keys():
    print(f"- {cat}")


🔎 Danh sách các category đã tạo bảng con:
- cmdi
- crypto
- hash
- ldapi
- pathtraver
- securecookie
- sqli
- trustbound
- weakrand
- xpathi
- xss


In [5]:
t = category["securecookie"]
t.show(5)
print("✅ True:", t.where("real_vulnerability", True).num_rows, 
      "| False:", t.where("real_vulnerability", False).num_rows)


test_name,category,real_vulnerability,cwe
BenchmarkTest00016,securecookie,False,614
BenchmarkTest00087,securecookie,True,614
BenchmarkTest00088,securecookie,False,614
BenchmarkTest00089,securecookie,False,614
BenchmarkTest00169,securecookie,True,614


✅ True: 36 | False: 31


## Đánh giá


In [ ]:
from datascience import Table

found_folder = "D:/VNU/Lab/Tools/zap_result/founded_testcase"
results = []

for filename in os.listdir(found_folder):
    if filename.endswith("_testcase.txt"):
        filepath = os.path.join(found_folder, filename)
        prefix = filename.replace("_testcase.txt", "")

        # 1) Đọc testcase từ file cleaned → D
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        D = set()
        for line in lines:
            line = line.strip()
            if line and not line.startswith("["):
                D.add(line)

        # 2) Lấy bảng con E từ category
        if prefix not in category:
            print(f"⚠️ Bỏ qua {filename} vì không tìm thấy category['{prefix}']")
            continue

        E = category[prefix]
        TP = FP = FN = TN = 0

        for t in E.rows:
            test_name = t[0]
            real_vuln = t[2]

            if test_name in D:
                if real_vuln:
                    TP += 1
                else:
                    FP += 1
            else:
                if real_vuln:
                    FN += 1
                else:
                    TN += 1

        print(f"[{filename}] ✅ TP:{TP} FP:{FP} FN:{FN} TN:{TN}")

        results.append([prefix, TP, FP, FN, TN])

# 3) Tạo bảng tổng hợp
final_result = Table().with_columns(
    "Category", [r[0] for r in results],
    "True Positive", [r[1] for r in results],
    "False Positive", [r[2] for r in results],
    "False Negative", [r[3] for r in results],
    "True Negative", [r[4] for r in results]
)

print("\n✅ Kết quả tổng hợp confusion matrix cho tất cả file cleaned:")
final_result.show()


[cmdi_testcase.txt] ✅ TP:67 FP:0 FN:59 TN:125
[pathtraver_testcase.txt] ✅ TP:15 FP:0 FN:118 TN:135
[securecookie_testcase.txt] ✅ TP:36 FP:4 FN:0 TN:27
[sqli_testcase.txt] ✅ TP:161 FP:9 FN:111 TN:223
[xss_testcase.txt] ✅ TP:137 FP:0 FN:109 TN:209

✅ Kết quả tổng hợp confusion matrix cho tất cả file cleaned:


Category,True Positive,False Positive,False Negative,True Negative
cmdi,67,0,59,125
pathtraver,15,0,118,135
securecookie,36,4,0,27
sqli,161,9,111,223
xss,137,0,109,209
